In [ ]:
import numpy as np
from numpy import random
from numpy import math
import matplotlib  
import matplotlib.pyplot as plt 

# copy
import copy


# import Keras and TF
import tensorflow as tf
from tensorflow import keras

from tensorflow.python.keras.models import clone_model
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Add, Dense, Activation, Flatten, Conv2D, Conv1D, MaxPooling2D, Dropout,BatchNormalization, Input, concatenate, Lambda
from tensorflow.python.keras.callbacks import Callback
from tensorflow.python.keras import regularizers
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator


from numpy import linalg as LA
from tensorflow.python.keras import backend as K

In [ ]:
# download the datasets

Path_trivial = 'data_AIII_expended\\trivial.npy'
Path_0 = 'data_AIII_expended\\w_0.npy'
Path_1 = 'data_AIII_expended\\w_1.npy'
Path_random = 'data_AIII_expended\\w_random.npy'

# data with label 0
data_0 = np.load(Path_trivial)

# data with label 1
data_1 = np.load(Path_1)

# the training dataset
N_train = round(0.95 * data_0.shape[0])

N_train_all = 2 * N_train

train_data = np.zeros((N_train_all, data_0.shape[1], data_0.shape[2]), dtype = float)
train_label = np.zeros((N_train_all), dtype = float)

train_data[:N_train, :, :] = data_0[:N_train, :, :]
train_data[N_train:, :, :] = data_1[:N_train, :, :]

train_label[:N_train] = np.zeros((N_train), dtype = float)
train_label[N_train:] = np.ones((N_train), dtype = float)


# the test dataset
N_test = data_0.shape[0] - round(0.95 * data_0.shape[0])

N_test_all = 2 * N_test

test_data = np.zeros((N_test_all, data_0.shape[1], data_0.shape[2]), dtype = float)
test_label = np.zeros((N_test_all), dtype = float)

test_data[:N_test, :, :] = data_0[N_train:, :, :]
test_data[N_test:, :, :] = data_1[N_train:, :, :]

test_label[:N_test] = np.zeros((N_test), dtype = float)
test_label[N_test:] = np.ones((N_test), dtype = float)

print(train_data.shape, train_label.shape)
print(test_data.shape, test_label.shape)

In [ ]:
def setup_network() : 
    
    # setup network 
   
    # don't use any regularization
    l2 = 0.000
    
    # setup cnn network
    
    input1 = Input(shape=(data_0.shape[1], data_0.shape[2]))  
 
    # convolution layers
    conv1_1 = Conv1D(128, kernel_size=(2), padding='valid', activation= 'relu', kernel_regularizer=regularizers.l2(l2), name='conv1_1')(input1)
 
    conv2_1 = Conv1D(64, kernel_size=(1), padding='valid', activation= 'relu', kernel_regularizer=regularizers.l2(l2), name='conv2_1')(conv1_1)
  
    conv3_1 = Conv1D(32, kernel_size=(1), padding='valid', activation= 'relu', kernel_regularizer=regularizers.l2(l2), name='conv3_1')(conv2_1)

    conv_all_1 = Conv1D(1, kernel_size=(1), padding='valid', activation= 'linear', kernel_regularizer=regularizers.l2(l2), name='conv_all_1')(conv3_1)
    
    flat = Flatten()(conv_all_1)
    
    # sum layer
    dense1 = Lambda( lambda x: tf.reshape(K.sum(x, axis = 1), (-1, 1)) , name='output1')(flat)

    model = Model(inputs=input1, outputs=dense1)

    
    return model


network = setup_network()

network.summary()

In [ ]:
# functions for doing 2d rotations: artificially expend the datasets during the training
def matrix_rot(phi):

    c = math.cos(phi)
    s = math.sin(phi)
    
    R = np.array([[ c, -s], [s,  c]])

    return R

def rotate_random(states):
    
    states_final = np.zeros((states.shape[0], states.shape[1], states.shape[2]), dtype = 'float')
    
    for n in range(states.shape[0]):
        
        # rotate over a random angle 'phi' around z-axis (in (-pi, pi))
        phi  = (2 * math.pi * random.random() - math.pi)
        matrix = matrix_rot(phi)
    
        for i_x in range(states.shape[1]):
            states_final[n, i_x, :] = np.matmul(matrix, states[n, i_x, :])

            
    return states_final
    



In [ ]:
# specify details for the training: learning rate 0.0001
my_adam = keras.optimizers.Adam(lr=0.00001)
network.compile(loss='mae', optimizer=my_adam, metrics=['accuracy'])

# train for 200 epoch 
for i in range(200):
    print('Epoch: ', i)
    
    # rotate the states to artificially expend the training dataset - avoid overfitting (optional)
    #train_data_rotated = rotate_random(train_data)
                                       
    network.fit(train_data, train_label, validation_data=(test_data, test_label), batch_size=512, epochs=1, shuffle=True)

    
# specify details for the training: learning rate 0.00001
my_adam = keras.optimizers.Adam(lr=0.00001)
network.compile(loss='mae', optimizer=my_adam, metrics=['accuracy'])

# train for 200 epoch 
for i in range(200):
    print('Epoch: ', i)
    
    # rotate the states to artificially expend the training dataset - avoid overfitting (optional)
    #train_data_rotated = rotate_random(train_data)
                                       
    network.fit(train_data, train_label, validation_data=(test_data, test_label), batch_size=512, epochs=1, shuffle=True)
        

In [ ]:
# Save the network

filepath = 'networks\\AIII_1d.h5'

network.save(filepath)

In [ ]:
from tensorflow.python.keras.models import load_model


filepath = 'networks\\AIII_1d.h5'

network = load_model(filepath)

In [ ]:
winding_num = network.predict(train_data)  
plt.hist(winding_num, 100)



In [ ]:
winding_num = network.predict(test_data)  
plt.hist(winding_num, 100)

In [ ]:
# create a dataset of random states (preprocessed in the same way as the training and test datasets)
def create_AIII_random(N_samples, N_k):
    
    data = np.zeros((N_samples, N_k + 1, 2), dtype = float)
    
    for n in range(N_samples):

        for i_x in range(N_k) : 
            
                
            h_x = random.random() - 0.5
            h_y = random.random() - 0.5

            E = (h_x**2 + h_y**2)**0.5
        
            data[n, i_x, 0] = h_x/E 
            data[n, i_x, 1] = h_y/E 
            
        #periodic boundary conditions
        data[n, N_k, :] = data[n, 0, :]
        
    return data 

# interpolate from N_k + 1 to 2*N_k + 1 k-points
def expend(data):
    
    N_k = data.shape[1] - 1 
    data_new = np.zeros((data.shape[0], 2*N_k+1, 2), dtype = float)
    
    for n in range(data.shape[0]):    
        
        for i_x in range(N_k + 1):
            data_new[n, 2*i_x, :] = data[n, i_x, :]
    
        for i_x in range(N_k): 
            data_new[n, 2*i_x + 1, :] = interpolate(data_new[n, 2*i_x, :], data_new[n, 2*i_x + 2, :])
       
  
    return data_new


# interpolation between two vectors          
def interpolate(H1, H2):
       
    H_int = (H1 + H2)/np.linalg.norm(H1 + H2)
    
    return H_int


N_samples = 1000
N_k = 100
data_random = expend(create_AIII_random(N_samples, N_k))

In [ ]:
# evaluate on a dataset of random states (observe quantization of the output)
winding_num = network.predict(data_random)  
plt.hist(winding_num, 100)